In [1]:
import os, sys, subprocess, json, time, pandas as pd, glob
from pathlib import Path

def run(cmd):
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    return p.stdout

print('=== GPU CHECK (nvidia-smi) ===', flush=True)
print(run(['bash','-lc','nvidia-smi || true']))

print('Python:', sys.version)
print('CWD:', os.getcwd())

root = Path('.')
print('\n=== Repo listing (top-level) ===')
for p in sorted(root.glob('*')):
    try:
        mt = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(p.stat().st_mtime))
        print(f"{p}  [mtime {mt}]")
    except Exception as e:
        print(p, e)

train_csv = Path('train.csv')
sample_sub = Path('sample_submission.csv')
print('\nFiles exist:', train_csv.exists(), sample_sub.exists())
df = pd.read_csv(train_csv)
print('train.csv shape:', df.shape)
print('train.csv head:\n', df.head())
print('Columns:', list(df.columns))

train_img_dir = Path('train_images')
test_img_dir = Path('test_images')
train_subdirs = sorted([p for p in train_img_dir.glob('*') if p.is_dir()])
test_images = sorted([p for p in test_img_dir.glob('*.jpg')])
print(f"train_images dirs: {len(train_subdirs)} | example: {train_subdirs[:3]}")
print(f"test_images count: {len(test_images)} | example: {test_images[:3]}")

print('\nLabel distribution (top 10):')
label_col = 'hotel_id' if 'hotel_id' in df.columns else df.columns[-1]
print(df[label_col].value_counts().head(10))

print('\nBasic checks done.')

=== GPU CHECK (nvidia-smi) ===


Sat Sep 27 05:40:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.06             Driver Version: 550.144.06     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q                 On  |   00000002:00:00.0 Off |                    0 |
| N/A   N/A    P0             N/A /  N/A  |     182MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Plan to Medal: Hotel-ID 2021 (FGVC8)

Objectives:
- Build a fast, strong baseline quickly; iterate to medal.
- Use GPU-accelerated image embeddings and retrieval; add training if needed.

Data Understanding:
- train.csv: 87,798 rows with columns [image, chain, hotel_id, timestamp].
- Images: train_images has 88 subdirs; test_images has 9,756 jpgs.
- Target: hotel_id; metric: MAP@5.

Validation Strategy:
- Start with quick holdout split stratified by hotel_id to validate pipeline.
- Upgrade to group-aware CV by hotel_id with temporal awareness: train < test-time split to mimic domain drift (avoid leakage via timestamp).
- Cache OOF predictions and embeddings.

Baseline v1 (No training, retrieval):
- Install Torch cu121 + timm/transformers/faiss-cpu.
- Use CLIP ViT-B/32 or ViT-L/14 pretrained encoder to extract embeddings for all train images.
- Build FAISS IndexFlatIP over L2-normalized train embeddings.
- For each test image, retrieve top-K nearest train images; aggregate hotel_id votes/similarities to produce top-5 hotels.
- Expect strong MAP@5 quickly; iterate on encoder choice and resolution.

Improvements v2:
- Try multiple backbones: CLIP ViT-L/14, ConvNeXt-T, EVA02-CLIP, NFNet. Blend embeddings (weighted).
- TTA: horizontal flip and multi-crop averaging of embeddings.
- Fine-tune with ArcFace/CosFace on hotel_id if time permits (timm + partial freeze).
- Use chain/time as priors or re-ranking (e.g., boost same-chain hotels).

Efficiency:
- Cache embeddings to .npy and metadata to .parquet.
- Log progress and elapsed time per batch.
- Subsample smoke test on 2k images before full run.

Submission:
- Create submission.csv with columns [image, hotel_id] where hotel_id is space-separated top-5 predictions.
- Verify format against sample_submission.csv head and length.

Next Steps (immediate):
1) Confirm image path resolution for train_images (map image name -> subdir).
2) Install Torch cu121 stack and required libs.
3) Implement embedding extraction script and cache train/test embeddings.
4) Implement FAISS retrieval, CV check, and produce a first submission.
5) Request expert review after baseline OOF and before full-scale compute/ensembles.

In [2]:
import os, sys, subprocess, shutil, time
from pathlib import Path

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# 0) Uninstall any preexisting torch stack to avoid mismatches
for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d, flush=True)
        shutil.rmtree(d, ignore_errors=True)

# 1) Install the EXACT cu121 torch stack FIRST
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# 2) Freeze torch versions
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

# 3) Install non-torch deps honoring constraints
pip('install', '-c', 'constraints.txt',
    'timm==1.0.9',
    'open_clip_torch==2.26.1',
    'faiss-cpu==1.7.4',
    'opencv-python-headless',
    'pillow',
    'pandas',
    'numpy',
    'scikit-learn',
    'tqdm',
    '--upgrade-strategy', 'only-if-needed')

# 4) Sanity check GPU
import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available'
print('GPU:', torch.cuda.get_device_name(0))

print('Environment ready.')

> install --index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://pypi.org/simple torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1


Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.org/simple


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 385.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 368.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 505.0 MB/s eta 0:00:00


...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 566.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 KB 428.5 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 KB 409.4 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 KB 421.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 KB 508.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 KB 468.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 KB 507.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 547.9 MB/s eta 0:00:00


torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: True
GPU: NVIDIA A10-24Q
Environment ready.


In [3]:
import pandas as pd
from pathlib import Path
import os, time

print('=== Build path mapping (train_images/{chain}/{image}) ===', flush=True)
train_csv = Path('train.csv')
df = pd.read_csv(train_csv)
df['chain'] = df['chain'].astype(str)
df['path'] = 'train_images/' + df['chain'] + '/' + df['image']

exists = df['path'].map(lambda p: Path(p).exists())
missing = (~exists).sum()
print(f'Total rows: {len(df)} | Missing files: {missing}')
if missing > 0:
    print('Example missing paths:', df.loc[~exists, 'path'].head(5).tolist())

# Save metadata cache
df[['image','chain','hotel_id','timestamp','path']].to_parquet('train_meta.parquet', index=False)
print('Saved train_meta.parquet')

print('\n=== Build test listing ===')
test_dir = Path('test_images')
test_files = sorted([p.name for p in test_dir.glob('*.jpg')])
test_df = pd.DataFrame({'image': test_files})
test_df['path'] = test_df['image'].map(lambda x: str(test_dir / x))
print('Test count:', len(test_df), '| head:', test_df.head().to_dict('records'))
test_df.to_parquet('test_meta.parquet', index=False)
print('Saved test_meta.parquet')

print('Path mapping complete.')

=== Build path mapping (train_images/{chain}/{image}) ===


Total rows: 87798 | Missing files: 0
Saved train_meta.parquet

=== Build test listing ===
Test count: 9756 | head: [{'image': '80196e6999ce63cf.jpg', 'path': 'test_images/80196e6999ce63cf.jpg'}, {'image': '80296afd55d516ea.jpg', 'path': 'test_images/80296afd55d516ea.jpg'}, {'image': '802aab95d62b7daa.jpg', 'path': 'test_images/802aab95d62b7daa.jpg'}, {'image': '802af4d04faf14df.jpg', 'path': 'test_images/802af4d04faf14df.jpg'}, {'image': '802b5ed622fd3587.jpg', 'path': 'test_images/802b5ed622fd3587.jpg'}]
Saved test_meta.parquet
Path mapping complete.


In [9]:
import os, time, math, gc
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image, ImageOps
import torch
import torch.nn.functional as F
from tqdm import tqdm
import open_clip

torch.backends.cudnn.benchmark = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def load_model():
    print('Loading OpenCLIP ViT-L/14-336 (openai)...', flush=True)
    model, _, preprocess = open_clip.create_model_and_transforms(
        'ViT-L-14-336', pretrained='openai'
    )
    model = model.to(device)
    model.eval()
    return model, preprocess

@torch.no_grad()
def embed_paths(paths, model, preprocess, batch_size=40, tta_hflip=False):
    embs = []
    n = len(paths)
    t0 = time.time()
    for i in range(0, n, batch_size):
        j = min(i + batch_size, n)
        batch_paths = paths[i:j]
        ims = []
        ims_flip = []
        for p in batch_paths:
            img = Image.open(p).convert('RGB')
            ims.append(preprocess(img))
            if tta_hflip:
                ims_flip.append(preprocess(ImageOps.mirror(img)))
        x = torch.stack(ims).to(device, non_blocking=True)
        with torch.amp.autocast('cuda', enabled=True):
            feat = model.encode_image(x)
        feat = F.normalize(feat.float(), dim=1)
        if tta_hflip:
            xf = torch.stack(ims_flip).to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=True):
                feat_f = model.encode_image(xf)
            feat_f = F.normalize(feat_f.float(), dim=1)
            feat = F.normalize((feat + feat_f) / 2, dim=1)
        embs.append(feat.cpu())
        if ((i // batch_size) % 20) == 0:
            elapsed = time.time() - t0
            print(f'  Embedded {j}/{n} images | elapsed {elapsed:.1f}s', flush=True)
        del x
        if tta_hflip:
            del xf
        torch.cuda.synchronize()
    embs = torch.cat(embs, dim=0).numpy().astype('float32')
    return embs

def run_embedding_pipeline(limit_train=None, limit_test=None, out_prefix='openclip_vitl14_336_openai_noTTA'):
    model, preprocess = load_model()
    tr = pd.read_parquet('train_meta.parquet')
    te = pd.read_parquet('test_meta.parquet')
    if limit_train is not None:
        tr = tr.sample(n=min(limit_train, len(tr)), random_state=42).reset_index(drop=True)
    if limit_test is not None:
        te = te.sample(n=min(limit_test, len(te)), random_state=42).reset_index(drop=True)
    print(f'Train images: {len(tr)} | Test images: {len(te)}')

    # Train embeddings
    t0 = time.time()
    tr_embs = embed_paths(tr['path'].tolist(), model, preprocess, batch_size=40, tta_hflip=False)
    np.save(f'{out_prefix}_train.npy', tr_embs)
    tr[['image','chain','hotel_id','path']].to_parquet(f'{out_prefix}_train_meta.parquet', index=False)
    print(f'Saved train embs: {tr_embs.shape} in {time.time()-t0:.1f}s')
    del tr_embs; gc.collect(); torch.cuda.empty_cache()

    # Test embeddings
    t1 = time.time()
    te_embs = embed_paths(te['path'].tolist(), model, preprocess, batch_size=40, tta_hflip=False)
    np.save(f'{out_prefix}_test.npy', te_embs)
    te[['image','path']].to_parquet(f'{out_prefix}_test_meta.parquet', index=False)
    print(f'Saved test embs: {te_embs.shape} in {time.time()-t1:.1f}s')

    print('All embeddings saved with prefix:', out_prefix)

# Control auto-run in this cell via SMOKE flag
SMOKE = False
if SMOKE:
    run_embedding_pipeline(limit_train=2000, limit_test=1000, out_prefix='openclip_vitl14_336_openai_noTTA_smoke')

In [10]:
import numpy as np
import pandas as pd
from pathlib import Path
import faiss
from collections import defaultdict
import os

def build_faiss_ip_index(embs: np.ndarray):
    assert embs.ndim == 2
    try:
        faiss.omp_set_num_threads(min(16, os.cpu_count() or 16))
    except Exception:
        pass
    index = faiss.IndexFlatIP(embs.shape[1])
    index.add(embs.astype('float32'))
    return index

def aggregate_sum_per_hotel(train_meta: pd.DataFrame, neighbor_idxs: np.ndarray, neighbor_sims: np.ndarray, topk: int = 500):
    scores = defaultdict(float)
    k = min(topk, len(neighbor_idxs))
    for ii, sim in zip(neighbor_idxs[:k], neighbor_sims[:k]):
        hid = int(train_meta.iloc[ii].hotel_id)
        s = max(float(sim), 0.0) ** 1.3
        scores[hid] += s
    return scores

def per_hotel_centroid_scores(q_vec: np.ndarray, train_meta: pd.DataFrame, train_embs: np.ndarray, neighbor_idxs: np.ndarray, topk: int = 500, cap_per_hotel: int = 60):
    # Build centroids from top-K neighbors per candidate hotel
    k = min(topk, len(neighbor_idxs))
    by_hotel = defaultdict(list)
    for ii in neighbor_idxs[:k]:
        hid = int(train_meta.iloc[ii].hotel_id)
        if len(by_hotel[hid]) < cap_per_hotel:
            by_hotel[hid].append(train_embs[ii])
    c_scores = {}
    for hid, vecs in by_hotel.items():
        V = np.stack(vecs).astype('float32')
        V /= (np.linalg.norm(V, axis=1, keepdims=True) + 1e-8)
        c = V.mean(axis=0)
        c /= (np.linalg.norm(c) + 1e-8)
        c_scores[hid] = float(np.dot(q_vec, c))
    return c_scores

def top5_from_scores(score_dict: dict):
    items = sorted(score_dict.items(), key=lambda x: -x[1])
    top = []
    seen = set()
    for hid, _ in items:
        s = str(hid)
        if s not in seen:
            top.append(s); seen.add(s)
        if len(top) == 5:
            break
    if not top:
        top = ['0']
    while len(top) < 5:
        top.append(top[-1])
    return ' '.join(top[:5])

def retrieve_and_submit(prefix: str, out_csv: str, K: int = 500, qe_m: int = 10, centroid_blend: float = 0.20):
    # Load embeddings and meta
    tr_emb = np.load(f'{prefix}_train.npy')
    te_emb = np.load(f'{prefix}_test.npy')
    tr_meta = pd.read_parquet(f'{prefix}_train_meta.parquet')
    te_meta = pd.read_parquet(f'{prefix}_test_meta.parquet')
    # Normalize
    tr_norm = tr_emb / (np.linalg.norm(tr_emb, axis=1, keepdims=True) + 1e-8)
    te_norm = te_emb / (np.linalg.norm(te_emb, axis=1, keepdims=True) + 1e-8)
    # Build FAISS
    print('Building FAISS IndexFlatIP with', tr_norm.shape, flush=True)
    index = build_faiss_ip_index(tr_norm.astype('float32'))
    K = min(K, tr_norm.shape[0])
    print('Searching...', flush=True)
    sims, idxs = index.search(te_norm.astype('float32'), K)
    # Aggregate per hotel with optional QE and centroid re-score
    preds = []
    for i in range(te_norm.shape[0]):
        q = te_norm[i]
        inds = idxs[i]
        s = sims[i]
        # Query expansion
        m = min(qe_m, len(inds))
        if m > 0:
            nn = tr_norm[inds[:m]]
            q2 = q + nn.mean(axis=0)
            q2 /= (np.linalg.norm(q2) + 1e-8)
        else:
            q2 = q
        # Sum of positives per hotel (ReLU^1.3)
        sum_scores = aggregate_sum_per_hotel(tr_meta, inds, s, topk=K)
        # Centroid per hotel from top-K and blend (cap=60)
        cent_scores = per_hotel_centroid_scores(q2, tr_meta, tr_norm, inds, topk=K, cap_per_hotel=60)
        # Merge
        final_scores = defaultdict(float)
        for hid, v in sum_scores.items():
            final_scores[hid] += (1.0 - centroid_blend) * v
        for hid, v in cent_scores.items():
            final_scores[hid] += centroid_blend * max(v, 0.0)
        preds.append(top5_from_scores(final_scores))
        if (i+1) % 200 == 0:
            print(f'  Scored {i+1}/{te_norm.shape[0]} queries', flush=True)
    # Align to required submission order
    sample = pd.read_csv('sample_submission.csv')
    sub = pd.DataFrame({'image': te_meta['image'].values, 'hotel_id': preds})
    sub = sample[['image']].merge(sub, on='image', how='left')
    # Fallback if needed
    if sub['hotel_id'].isna().any():
        freq = tr_meta['hotel_id'].value_counts().index.astype(str).tolist()[:5]
        fallback = ' '.join(freq + [freq[-1]]*(5-len(freq)))
        sub['hotel_id'] = sub['hotel_id'].fillna(fallback)
    assert sub['hotel_id'].str.split().map(len).eq(5).all(), 'Each row must have 5 predictions'
    sub.to_csv(out_csv, index=False)
    print('Saved submission to', out_csv)

print('Retrieval functions ready. Use retrieve_and_submit with prefix from cell 4 output.')

Retrieval functions ready. Use retrieve_and_submit with prefix from cell 4 output.


In [ ]:
# Full extraction (ViT-L/14-336 openai, no TTA) + retrieval to submission.csv
import time, os
t0 = time.time()
print('=== Starting FULL embedding extraction (ViT-L/14-336 openai, no TTA) ===', flush=True)
FULL_PREFIX = 'openclip_vitl14_336_openai_noTTA'
try:
    # Extract embeddings for all train/test
    run_embedding_pipeline(limit_train=None, limit_test=None, out_prefix=FULL_PREFIX)
    print(f'Embedding extraction done in {time.time()-t0:.1f}s, starting retrieval...', flush=True)
    # Retrieval with similarity-sum + QE + centroid re-score
    retrieve_and_submit(prefix=FULL_PREFIX, out_csv='submission.csv', K=200, qe_m=5, centroid_blend=0.3)
    print('submission.csv created. Verify head and length match sample_submission.csv')
except NameError as e:
    print('Functions not found in current kernel. Please re-run cells 4 and 5 first.', e)

=== Starting FULL embedding extraction (ViT-L/14-336 openai, no TTA) ===


Loading OpenCLIP ViT-L/14-336 (openai)...


  0%|                                               | 0.00/934M [00:00<?, ?iB/s]

  0%|                                       | 164k/934M [00:00<10:10, 1.53MiB/s]

  0%|                                       | 360k/934M [00:00<09:12, 1.69MiB/s]

...

 15%|█████▋                                 | 136M/934M [00:03<00:13, 57.2MiB/s]

 16%|██████▏                                | 148M/934M [00:03<00:11, 71.3MiB/s]

 17%|██████▌                                | 156M/934M [00:04<00:33, 23.3MiB/s]

 17%|██████▊                                | 162M/934M [00:04<00:29, 25.7MiB/s]

 18%|███████▏                               | 173M/934M [00:04<00:21, 35.5MiB/s]

In [8]:
# Install FAISS-CPU (fallback) and sanity check
import sys, subprocess

def pip(*args):
    print('>', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

print('Installing faiss-cpu (GPU wheels unavailable in this environment)...')
pip('install', 'faiss-cpu==1.7.4')

import faiss, numpy as np
print('FAISS version:', faiss.__version__)
d = 64
index = faiss.IndexFlatIP(d)
xb = np.random.randn(1000, d).astype('float32')
xb /= np.linalg.norm(xb, axis=1, keepdims=True) + 1e-8
index.add(xb)
xq = np.random.randn(10, d).astype('float32')
xq /= np.linalg.norm(xq, axis=1, keepdims=True) + 1e-8
D, I = index.search(xq, 5)
print('FAISS-CPU sanity OK. Top5 shape:', I.shape)

Installing faiss-cpu (GPU wheels unavailable in this environment)...
> install faiss-cpu==1.7.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 167.6 MB/s eta 0:00:00


FAISS version: 1.7.4
FAISS-CPU sanity OK. Top5 shape: (10, 5)


In [ ]:
# Temporal per-hotel CV (holdout newest 20%) using precomputed train embeddings
import numpy as np
import pandas as pd
import faiss
from collections import defaultdict
from datetime import datetime
import time

def parse_ts(s):
    return pd.to_datetime(s)

def build_temporal_holdout(df: pd.DataFrame, val_frac: float = 0.2, min_keep_train: int = 3):
    df = df.copy()
    df['ts'] = parse_ts(df['timestamp'])
    tr_idx, val_idx = [], []
    for hid, g in df.groupby('hotel_id'):
        g = g.sort_values('ts')
        n = len(g)
        if n < min_keep_train:
            tr_idx.extend(g.index.tolist())
            continue
        cut = max(1, int(round(n * (1 - val_frac))))
        tr_idx.extend(g.index[:cut].tolist())
        val_idx.extend(g.index[cut:].tolist())
    return np.array(tr_idx, dtype=int), np.array(val_idx, dtype=int)

def l2norm(x):
    return x / (np.linalg.norm(x, axis=1, keepdims=True) + 1e-8)

def map_at_5(truth: np.ndarray, preds: list):
    score = 0.0
    for t, p in zip(truth, preds):
        parts = p.split()[:5]
        for k, h in enumerate(parts, 1):
            if str(t) == h:
                score += 1.0 / k
                break
    return score / len(truth)

def cv_evaluate(prefix: str, K: int = 200, qe_m: int = 5, centroid_blend: float = 0.3, val_frac: float = 0.2):
    # Load full train embeddings and meta
    tr_emb = np.load(f'{prefix}_train.npy')
    meta = pd.read_parquet(f'{prefix}_train_meta.parquet')
    tr_emb = tr_emb.astype('float32')
    tr_emb = l2norm(tr_emb)
    tr_idx_all, val_idx_all = build_temporal_holdout(meta, val_frac=val_frac, min_keep_train=3)
    print(f'Train split size: {len(tr_idx_all)} | Val split size: {len(val_idx_all)}', flush=True)
    # Build FAISS on train-only
    dim = tr_emb.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(tr_emb[tr_idx_all])
    # Query = val embeddings
    q = tr_emb[val_idx_all]
    Kq = min(K, len(tr_idx_all))
    print('Searching val queries...', flush=True)
    sims, idxs = index.search(q, Kq)
    # Map back to global train indices
    neigh_global = tr_idx_all[idxs]
    # Remove self-matches (if any same image in gallery)
    preds = []
    y_true = meta.iloc[val_idx_all]['hotel_id'].values
    # Reuse retrieval helpers from cell 5
    for i in range(q.shape[0]):
        inds = neigh_global[i]
        sims_i = sims[i]
        # Filter exact self index if present
        mask = inds != val_idx_all[i]
        inds = inds[mask]
        sims_f = sims_i[mask]
        # QE
        m = min(qe_m, len(inds))
        if m > 0:
            nn = tr_emb[inds[:m]]
            q2 = q[i] + nn.mean(axis=0)
            q2 /= (np.linalg.norm(q2) + 1e-8)
        else:
            q2 = q[i]
        # Sum positives
        sum_scores = aggregate_sum_per_hotel(meta, inds, sims_f, topk=Kq)
        # Centroid blend
        cent_scores = per_hotel_centroid_scores(q2, meta, tr_emb, inds, topk=Kq, cap_per_hotel=50)
        final_scores = defaultdict(float)
        for hid, v in sum_scores.items():
            final_scores[hid] += (1.0 - centroid_blend) * v
        for hid, v in cent_scores.items():
            final_scores[hid] += centroid_blend * max(v, 0.0)
        preds.append(top5_from_scores(final_scores))
        if (i+1) % 500 == 0:
            print(f'  Val scored {i+1}/{q.shape[0]}', flush=True)
    m5 = map_at_5(y_true, preds)
    print(f'CV MAP@5 (val newest {int(val_frac*100)}%): {m5:.5f}')
    return m5

print('Temporal CV utilities ready. After embeddings complete, run cv_evaluate("openclip_vitl14_336_openai_noTTA") to validate and tune K/centroid_blend.')

In [ ]:
# Regenerate submission with tuned retrieval params after embeddings finish
print('Rebuilding submission with K=500, QE m=10, centroid_blend=0.20, cap_per_hotel=60, ReLU^1.3', flush=True)
prefix = FULL_PREFIX if 'FULL_PREFIX' in globals() else 'openclip_vitl14_336_openai_noTTA'
retrieve_and_submit(prefix=prefix, out_csv='submission.csv', K=500, qe_m=10, centroid_blend=0.20)
import pandas as pd, os, time
sub = pd.read_csv('submission.csv')
print('submission.csv shape:', sub.shape, '| head:\n', sub.head())
print('mtime:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(os.path.getmtime('submission.csv'))), flush=True)